In [2]:
import os, sys, glob, pickle, tables

import numpy as np
# pip install simweights <--- run this once to install the package
# then replace <username> with your username and <version> with your Python version
sys.path.append('/home/ahinners/.local/lib/python3.11/site-packages')
import simweights

In [3]:
# Load in IceCube CORSIKA Simulations
bigfilelist = '/data/sim/IceCube/2020/filtered/dst/CORSIKA-in-ice/20904'
corsika_fl = sorted(glob.glob(f'{bigfilelist}/*/*i3*'))
#0000000-0000999
corsika_fl = corsika_fl[:50]

In [4]:
weight_keys = [
    "CylinderLength",
    "CylinderRadius",
    "EnergyPrimaryMax",
    "EnergyPrimaryMin",
    "NEvents",
    "OverSampling",
    "ParticleType",
    "PrimaryEnergy",
    "PrimarySpectralIndex",
    "PrimaryType",
    "ThetaMax",
    "ThetaMin",
    "Weight",
]

particle_keys = ["type", "energy", "zenith"]

CorsikaWeightMap: dict = {k: [] for k in weight_keys}
PolyplopiaPrimary: dict = {k: [] for k in particle_keys}
MCtype_corsika = np.array([])
MCenergy_corsika = np.array([])

In [ ]:
for f in corsika_fl:
    #print("Reading", f)
    infile_corsika = dataio.I3File(f)
    while infile_corsika.more():
        frame = infile_corsika.pop_physics()
        
        # Frame may contain coincident events so select injected primary shower 'PolyplopiaPrimary'
        MCtype_corsika = np.append(MCtype_corsika, frame["PolyplopiaPrimary"].type)
        MCenergy_corsika = np.append(MCenergy_corsika, frame["PolyplopiaPrimary"].energy)

        for k in weight_keys:
            CorsikaWeightMap[k].append(frame["CorsikaWeightMap"][k])

        PolyplopiaPrimary["zenith"].append(frame["PolyplopiaPrimary"].dir.zenith)
        PolyplopiaPrimary["type"].append(frame["PolyplopiaPrimary"].type)
        PolyplopiaPrimary["energy"].append(frame["PolyplopiaPrimary"].energy)

In [ ]:
# Set the weighter for the data
fobj = {"CorsikaWeightMap": CorsikaWeightMap, "PolyplopiaPrimary": PolyplopiaPrimary, "DST_Cuts": DST_Cuts}
wobj = simweights.CorsikaWeighter(fobj, nfiles=len(corsika_fl))

In [ ]:
PICKLING_FILES = {
    'energy': weighter.get_column('PolyplopiaPrimary', 'energy'),
    'type': weighter.get_column('PolyplopiaPrimary', 'type'),
    'zenith': weighter.get_column('PolyplopiaPrimary', 'zenith'),
    'Nevents': weighter.get_column('weight_keys', 'NEvents'),
    'fluxH4a': wobj.get_weights(simweights.GaisserH4a())
    'fluxGSF': wobj.get_weights(simweights.GlobalSplineFit())
}

for key, array in PICKLING_FILES.items():
    with open(f'pickles/{key}.pkl', 'wb') as f:
        pickle.dump(array, f)